# Palmer Penguins Modeling

Import the Palmer Penguins dataset and print out the first few rows.

Suppose we want to predict `bill_depth_mm` using the other variables in the dataset.

**Dummify** all variables that require this.

In [2]:
import pandas as pd
import numpy as np
from palmerpenguins import load_penguins
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score

In [1]:
!pip install palmerpenguins

from palmerpenguins import load_penguins
penguins = load_penguins()
penguins.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007


In [7]:
penguins_df = pd.get_dummies(penguins)
penguins_df

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,year,species_Adelie,species_Chinstrap,species_Gentoo,island_Biscoe,island_Dream,island_Torgersen,sex_female,sex_male
0,39.1,18.7,181.0,3750.0,2007,True,False,False,False,False,True,False,True
1,39.5,17.4,186.0,3800.0,2007,True,False,False,False,False,True,True,False
2,40.3,18.0,195.0,3250.0,2007,True,False,False,False,False,True,True,False
3,NaN,NaN,NaN,NaN,2007,True,False,False,False,False,True,False,False
4,36.7,19.3,193.0,3450.0,2007,True,False,False,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,55.8,19.8,207.0,4000.0,2009,False,True,False,False,True,False,False,True
340,43.5,18.1,202.0,3400.0,2009,False,True,False,False,True,False,True,False
341,49.6,18.2,193.0,3775.0,2009,False,True,False,False,True,False,False,True
342,50.8,19.0,210.0,4100.0,2009,False,True,False,False,True,False,False,True


Let's use the other variables to predict `bill_depth_mm`. Prepare your data and fit the following models on a training dataset subset of the entire dataset:

* Four different models, each containing a different set of predictor variables

Create a plot like the right plot of Fig 1. in our `Model Validation` chapter with the training and test error plotted for each of your four models.

Which of your models was best?

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [16]:
cleaned_penguins = penguins_df.dropna()

y = cleaned_penguins['bill_depth_mm']

# Define feature sets for each model
model_one_features = ["bill_length_mm"]
model_two_features = ["bill_length_mm", "flipper_length_mm"]
model_three_features = ["bill_length_mm", "flipper_length_mm", "body_mass_g"]
model_four_features = [col for col in cleaned_penguins.columns if col != "bill_depth_mm"]

# Select predictors for each model
X_model1 = cleaned_penguins[model_one_features]
X_model2 = cleaned_penguins[model_two_features]
X_model3 = cleaned_penguins[model_three_features]
X_model4 = cleaned_penguins[model_four_features]

# Split data for each model
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_model1, y, test_size=0.25, random_state=42)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_model2, y, test_size=0.25, random_state=42)
X_train3, X_test3, y_train3, y_test3 = train_test_split(X_model3, y, test_size=0.25, random_state=42)
X_train4, X_test4, y_train4, y_test4 = train_test_split(X_model4, y, test_size=0.25, random_state=42)

# Fit linear regression models
model_one = LinearRegression().fit(X_train1, y_train1)
model_two = LinearRegression().fit(X_train2, y_train2)
model_three = LinearRegression().fit(X_train3, y_train3)
model_four = LinearRegression().fit(X_train4, y_train4)

# Make predictions
y_pred_train1, y_pred_test1 = model_one.predict(X_train1), model_one.predict(X_test1)
y_pred_train2, y_pred_test2 = model_two.predict(X_train2), model_two.predict(X_test2)
y_pred_train3, y_pred_test3 = model_three.predict(X_train3), model_three.predict(X_test3)
y_pred_train4, y_pred_test4 = model_four.predict(X_train4), model_four.predict(X_test4)

In [18]:
# Calculate RMSE scores
train_rmse_scores = [
    np.sqrt(mean_squared_error(y_train1, y_pred_train1)),
    np.sqrt(mean_squared_error(y_train2, y_pred_train2)),
    np.sqrt(mean_squared_error(y_train3, y_pred_train3)),
    np.sqrt(mean_squared_error(y_train4, y_pred_train4))]

test_rmse_scores = [
    np.sqrt(mean_squared_error(y_test1, y_pred_test1)),
    np.sqrt(mean_squared_error(y_test2, y_pred_test2)),
    np.sqrt(mean_squared_error(y_test3, y_pred_test3)),
    np.sqrt(mean_squared_error(y_test4, y_pred_test4))]

In [20]:
model_names = ["Model 1", "Model 2", "Model 3", "Model 4"]

plot_data_rmse = pd.DataFrame({
    'Model': model_names * 2,
    'RMSE': train_rmse_scores + test_rmse_scores,
    'Dataset': ['Training'] * len(train_rmse_scores) + ['Test'] * len(test_rmse_scores)
})

plot_data_rmse

,Model,RMSE,Dataset
0,Model 1,1.908203,Training
1,Model 2,1.549351,Training
2,Model 3,1.547310,Training
3,Model 4,0.778012,Training
4,Model 1,1.946328,Test
5,Model 2,1.568472,Test
6,Model 3,1.557992,Test
7,Model 4,0.785412,Test
